## 1. Import Library dan Model

In [1]:
#import library normal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# metric classification
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,confusion_matrix, f1_score, roc_curve, precision_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score 


#import Sklearn
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler,OneHotEncoder,OrdinalEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

#save
import pickle
import json
import warnings
warnings.filterwarnings("ignore")

In [92]:
#Load All Files
with open('model_gbc.pkl', 'rb') as file_1:
  model_gbc = pickle.load(file_1)

with open('model_numeric_transformer.pkl', 'rb') as file_2:
  model_numeric_transfomer = pickle.load(file_2)

with open('model_categoric_transformer.pkl', 'rb') as file_3:
  model_categoric_transfomer = pickle.load(file_3)

with open('list_num_columns_fs.txt', 'r') as file_4:
  list_num_columns_fs = json.load(file_4)

with open('list_cat_columns_fs.txt', 'r') as file_5:
  list_cat_columns_fs = json.load(file_5)

with open('model_xgb_tuned.pkl', 'rb') as file_6:
  model_xgb_tuned = pickle.load(file_6)

## 2. Load Data

In [142]:
df_inf=pd.read_csv("application_test.csv")
df_inf

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


## 3. Predict

In [143]:
df_inf = df_inf[df_inf["REGION_RATING_CLIENT_W_CITY"] != -1]

In [144]:
df_inf.dropna(subset=["NAME_TYPE_SUITE"], inplace=True)
df_inf.dropna(subset=["OCCUPATION_TYPE"], inplace=True)

In [145]:
df_inf.reset_index(drop=True, inplace=True)
df_inf

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100042,Cash loans,F,Y,Y,0,270000.0,959688.0,34600.5,810000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
3,100057,Cash loans,M,Y,Y,2,180000.0,499221.0,22117.5,373500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,100065,Cash loans,M,N,Y,0,166500.0,180000.0,14220.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,456168,Cash loans,F,N,Y,0,135000.0,835686.0,46660.5,765000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
32557,456189,Cash loans,F,N,Y,0,270000.0,360000.0,28570.5,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
32558,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
32559,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [146]:
df_inf.shape

(32561, 121)

In [147]:
df_inf_num=df_inf[list_num_columns_fs]
df_inf_cat=df_inf[list_cat_columns_fs]

In [148]:
# Feature Scaling
df_inf_num_scaled = model_numeric_transfomer.transform(df_inf_num)
# Feature Encoded
df_inf_cat_encoded = model_categoric_transfomer.transform(df_inf_cat).toarray()

In [149]:
#concat antara df scaled dan non scaled
df_inf_final = np.concatenate([df_inf_num_scaled,df_inf_cat_encoded], axis=1)

In [150]:
#Predict using XGB
y_pred_inf = model_xgb_tuned.predict(df_inf_final)
y_pred_inf

array([0, 0, 1, ..., 1, 0, 1])

In [152]:
df_pred = pd.DataFrame({'TARGET': y_pred_inf})
df_pred

,TARGET
0,0
1,0
2,1
3,0
4,0
...,...
32556,0
32557,1
32558,1
32559,0


In [154]:
df_concat = pd.concat([df_pred, df_inf], axis=1)
df_concat

,TARGET,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,0,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,1,100042,Cash loans,F,Y,Y,0,270000.0,959688.0,34600.5,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
3,0,100057,Cash loans,M,Y,Y,2,180000.0,499221.0,22117.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,100065,Cash loans,M,N,Y,0,166500.0,180000.0,14220.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0,456168,Cash loans,F,N,Y,0,135000.0,835686.0,46660.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
32557,1,456189,Cash loans,F,N,Y,0,270000.0,360000.0,28570.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
32558,1,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
32559,0,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [157]:
#ambil data seperti sample_submission
list_submission=["SK_ID_CURR","TARGET"]
df_submission=df_concat[list_submission]
# Simpan DataFrame ke file CSV
df_submission.to_csv('submission.csv', index=False)